In [1]:
import numpy as np
import pandas as pd
import operator
import pickle
import shelve
import re
from collections import Counter, defaultdict, OrderedDict, deque
import csv
import rdflib
from rdflib.namespace import OWL, RDF, RDFS
from rdflib import URIRef
import os
import string
import math

In [2]:
#####################################################
################ Initialization #####################
#####################################################

#ground truth metadata file
labeledFile = 'ebu3b_sensor_types_location.csv'
with open(labeledFile, 'rb') as fp:
    #Import list of points to a data frame
    #Pandas data frame allows you to store records and manipulate them: http://pandas.pydata.org/
    points_list = pd.DataFrame.from_csv(fp)
    points_list = points_list.set_index(keys='Unique Identifier')
    points_list.index.str.strip()
    equip_subsystem = points_list['Equipment Subsystems'] #saving to preserve formatting
    points_list = points_list.replace(r'\s+|-', '_', regex=True)
    
#Import lookup table that maps EBU3B descriptions to Brick tagsets
desc_tagset_map = pd.read_csv('ebu3b_description_to_tagset.csv')
desc_tagset_map['BACnet Description'] = desc_tagset_map['BACnet Description'].str.lower()
desc_tagset_map = desc_tagset_map.replace(r'\s+|-','_',regex=True).set_index('BACnet Description').dropna()

In [3]:
#####################################################
######## Initialize RDFlib graph and namespaces #####
#####################################################

#Namespaces
#Note: The URLs are fake
BRICK = rdflib.Namespace('http://buildsys.org/ontologies/Brick#')
BRICKFRAME = rdflib.Namespace('http://buildsys.org/ontologies/BrickFrame#')
EBU3B = rdflib.Namespace('http://ucsd.edu/building/ontology/ebu3b#')
#RDF, RDFS and OWL have already been imported in the library initializations

#Initiate graph from base ttl file
building_graph = rdflib.Graph()
brick_graph = rdflib.Graph()
building_graph.bind('ebu3b', EBU3B)
building_graph.bind('brick', BRICK)
building_graph.bind('brickframe', BRICKFRAME)
building_graph.bind('owl', OWL)
brick_graph.bind('brick', BRICK)
brick_graph.bind('brickframe',BRICKFRAME)
brick_graph.parse('../../BuildingSchema/Brick.ttl', format='turtle')
#building_graph.parse('../../BuildingSchema/Brick_Protege.owl', format='turtle')
#brick_graph.serialize(destination='brick_rdflib.ttl', format='turtle')

<Graph identifier=N8a46340fb9f74ab8b1148e7c834089ff (<class 'rdflib.graph.Graph'>)>

In [4]:
relationships_count = 0

#Adding Location information
#Convert location to title case
points_list['Location'] = points_list['Location'].apply((lambda x: x.title() if isinstance(x, basestring) else x))
location_list = points_list.reset_index().Location.drop_duplicates().dropna()
location_list = list(location_list)
location_list = [x for x in location_list if 'xxx' not in x]

#Add building and floors
building_graph.add((EBU3B.EBU3B, RDF.type, BRICK.Building))
building_graph.add((EBU3B.EBU3B_Basement, RDF.type, BRICK.Floor))
building_graph.add((EBU3B.EBU3B_Floor_1, RDF.type, BRICK.Floor))
building_graph.add((EBU3B.EBU3B_Floor_2, RDF.type, BRICK.Floor))
building_graph.add((EBU3B.EBU3B_Floor_3, RDF.type, BRICK.Floor))
building_graph.add((EBU3B.EBU3B_Floor_4, RDF.type, BRICK.Floor))
building_graph.add((EBU3B.EBU3B_Basement, BRICKFRAME.isPartOf, EBU3B.EBU3B))
building_graph.add((EBU3B.EBU3B_Floor_1, BRICKFRAME.isPartOf, EBU3B.EBU3B))
building_graph.add((EBU3B.EBU3B_Floor_2, BRICKFRAME.isPartOf, EBU3B.EBU3B))
building_graph.add((EBU3B.EBU3B_Floor_3, BRICKFRAME.isPartOf, EBU3B.EBU3B))
building_graph.add((EBU3B.EBU3B_Floor_4, BRICKFRAME.isPartOf, EBU3B.EBU3B))

relationships_count += 5

for room in location_list:
    #convert to title case
    #room = room.title()
    room_label = EBU3B["EBU3B_"+room]
    zone_label = EBU3B["EBU3B_HVAC_Zone_"+room]
    #add room as HVAC Zone
    building_graph.add((zone_label, RDF.type, BRICK.HVAC_Zone))
    #get floor
    floor = "EBU3B_Floor_" + room[3] if "Rm" in room else None
    if floor:
        floor = "EBU3B_Basement" if "Floor_B" in floor else floor
        building_graph.add((zone_label, BRICKFRAME.isPartOf, EBU3B[floor]))
        relationships_count += 1
    #TODO: Add rooms contained in a zone
    building_graph.add((room_label, RDF.type, BRICK.Room))
    building_graph.add((room_label, BRICKFRAME.isPartOf, zone_label))
    relationships_count += 1

In [5]:
#Load all equipment to building graph
#Create equipment id
points_list['equip_id'] = points_list['Building'] + '_' + equip_subsystem.replace(r' -> ', '_', regex=True)
points_list.equip_id = points_list.equip_id.replace(r'\s+', '_', regex=True)

#Get equipment related terms
equip_list = points_list[['Building','Equipment Type','Equipment Ref','Location','equip_id']]
equip_list['equip_subsystem'] = equip_subsystem 

#print equip_list.equip_id.head()
#print equip_list[equip_list.equip_id.str.contains('Hot_Water_System')].equip_id
    
#Drop duplicates to get list of equipment only
equip_list = equip_list.set_index('equip_id').drop_duplicates()

#Iterate through list and add to schema
for equip_id, equip_metadata in equip_list.iterrows():
    #check for nan and empty strings
    if equip_id != equip_id or equip_id.strip() == '':
        continue
    #Add EBU3B prefix
    equip_label = EBU3B[equip_id]
    #Add to schema
    #building_graph.add((equip_label, RDF.type, OWL.NamedIndividual))
    
    #Add equipment type
    equip_type = str(equip_metadata['Equipment Type'])
    if equip_type == 'nan': #skip blanks
        continue
    if 'CRAC' in equip_type: #CRAC formatting
        equip_type = 'CRAC'
    equip_type = BRICK[equip_type]
    if (equip_type, None, None) in brick_graph:
        #Add equip type to schema
        building_graph.add((equip_label, RDF.type, equip_type))
    else:
        print "Could not identify equipment type in schema:", equip_type
        building_graph.add((equip_label, RDF.type, BRICK.Equipment))
        
    #All the VAVs are fed by AHU_1, and VAVs feed their corresponding HVAC Zones
    if "VAV" in equip_id and not(any(x in equip_id for x in ["Thermostat","Damper","Valve","Fan","Unknown"])):
        building_graph.add((EBU3B.EBU3B_AHU_1, BRICKFRAME.feeds, equip_label))
        
        zone_id = equip_id.replace("VAV", "HVAC_Zone")
        building_graph.add((equip_label, BRICKFRAME.feeds, EBU3B[zone_id]))

/Users/bbalaji/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
#Add equipment isPartOf relationships
for equip_id, equip_metadata in equip_list.iterrows():
    #check for nan and empty strings
    if equip_id != equip_id or equip_id.strip() == '':
        continue
    #Parse the equipment hierarchy from CSV file
    equip_parts = equip_metadata['equip_subsystem'].split(' -> ')
    equip_hier = []
    for i,part in enumerate(equip_parts):
        part = '_'.join(part.split())
        part_ref = '_'.join(equip_parts[i+1].split()) if i+1 < len(equip_parts) else None
        if (BRICK[part],None, None) in brick_graph:
            if (BRICK[part_ref],None,None) in brick_graph or part_ref == None: 
                part_id = part
                equip_ref = equip_id.split(part_id)[0] + part_id
                equip_hier.append(equip_ref)
            else:
                part_id = part + '_' + part_ref
                equip_ref = equip_id.split(part_id)[0] + part_id
                equip_hier.append(equip_ref)
    equip_hier.reverse()
    #print equip_hier
    for i,equip_id in enumerate(equip_hier):
        if i+1 < len(equip_hier):
            building_graph.add((EBU3B[equip_id], BRICKFRAME.isPartOf, EBU3B[equip_hier[i+1]]))
            #print equip_id, "isPartOf", equip_hier[i+1]
        

In [7]:
#Load all points to the building instance
point_count = 0
point_tagset_count = 0
nan_count = 0
no_desc_count = 0
no_point_type_count = 0
for point_id, point_metadata in points_list.iterrows():
    #Create point label
    point_label = re.sub('\.+','_',point_metadata['Johnson Controls Name'])
    #Add building prefix
    point_label = EBU3B[point_label]
    #Add to Schema
    #building_graph.add((point_label, RDF.type, OWL.NamedIndividual))
    point_count += 1

    #Adding Point Type
    #Look up the schema tagset
    desc = point_metadata['BACnet Description']
    if str(desc) == 'nan':
        nan_count += 1
        print "Found nan as description:", point_label
        continue
    desc = desc.lower()
    if desc in desc_tagset_map.index:
        point_type = desc_tagset_map.loc[desc]['Schema Label']
    else:
        no_desc_count += 1
        print "No description match found:", point_label, desc
        continue
    if isinstance(point_type, pd.Series):
        point_type = point_type[0]
    if str(point_type) == 'nan':
        nan_count += 1
        print "Found nan as point type:", point_label
        continue
    #Get the first match, capitalize the first letter of each word and concatenate by '_'
    #print point_type
    point_type = '_'.join([word[0].upper() + word[1:] for word in point_type.split('_')])
    #Take care of Synonyms
    if 'Supply Air' in point_type: 
        point_type = point_type.replace('Supply Air','Discharge Air')
    if 'Co2' in point_type:
        point_type = point_type.replace('Co2', 'CO2')
    #Concatenate with brick prefix
    point_type = BRICK[point_type]
    #print point_type
    #Check if point type exists in the schema
    if (point_type, None, None) in brick_graph:
        #Add point type to schema
        building_graph.add((point_label, RDF.type, point_type))
        point_tagset_count += 1
    else:
        print "Could not identify point type in schema:", point_type 
        no_point_type_count += 1
        building_graph.add((point_label, RDF.type, BRICK.Point))
        
    #Add isPointOf relationship
    equip_id = point_metadata['equip_id']
    if equip_id != equip_id or equip_id.strip() == '':
        continue
    equip_label = EBU3B[equip_id]
    #Add to schema
    building_graph.add((point_label, BRICKFRAME.isPointOf, equip_label))
    #Add transitive isPointOf for VAV and AHU #can be removed, depending on app requirements
    if "AHU" in equip_id:
        equip_id = '_'.join(equip_id.split('_')[:3])
        building_graph.add((point_label, BRICKFRAME.isPointOf, EBU3B[equip_id]))
    elif "VAV" in equip_id and "unknown" not in equip_id.lower():
        equip_id = '_'.join(equip_id.split('_')[:4])
        building_graph.add((point_label, BRICKFRAME.isPointOf, EBU3B[equip_id]))
    #Add thermostat points to the zone
    equip_id = point_metadata['equip_id']
    if "Thermostat" in equip_id and "Unknown" not in equip_id:
        zone_label = EBU3B["EBU3B_HVAC_Zone_"+point_metadata["Location"]]
        building_graph.add((point_label, BRICKFRAME.isPointOf, zone_label))
        #print point_label, 'isPointOf', zone_label

print point_count, point_tagset_count, nan_count, no_desc_count, no_point_type_count

Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://builds

In [8]:
print desc_tagset_map.loc['occupied_command']

BACnet Type Str            Multi_State_Output
BACnet Unit                           unknown
Ground Truth Point Type     Occupancy_Command
Schema Label                Occupancy_Command
Name: occupied_command, dtype: object


In [9]:
#Add power meters
electrical_power_meters = [
    {"Main_A":"EBU3B"},
    {"Main_B":"EBU3B"},
    {"Elevator":"EBU3B_Elevator"},
    {"Server_Room":"EBU3B_Server_Room"},
    {"HVAC":"EBU3B_HVAC"},
    {"Lighting_Floor_1": None}, #"EBU3B_Lighing_Panel_Floor_1"},
    {"Lighting_Floor_2": None}, #"EBU3B_Lighing_Panel_Floor_2"},
    {"Lighting_Floor_3": None}, #"EBU3B_Lighing_Panel_Floor_3"},
    {"Lighting_Floor_4": None}, #"EBU3B_Lighing_Panel_Floor_4"},
    {"Lighting_Basement": None}, #"EBU3B_Lighing_Panel_Basement"}
]

cooling_thermal_power_meters = [
    "HVAC",
    "CRAC"
]

heating_thermal_power_meters = [
    "HVAC",
    "CRAC"
]

#Add missing equipment, location
building_graph.add((EBU3B.EBU3B_Elevator, RDF.type, BRICK.Elevator))

for meter in electrical_power_meters:
    meter_id = meter.keys()[0]
    meter_point_of = meter.values()[0]
    meter_label = "EBU3B_Power_Meter_" + meter_id
    building_graph.add((EBU3B[meter_label], RDF.type, BRICK.Electrical_Power_Meter))
    if meter_point_of:
        building_graph.add((EBU3B[meter_label], BRICK.isPointOf, EBU3B[meter_point_of]))
        
for meter in cooling_thermal_power_meters:
    meter_label = "EBU3B_Cooling_Thermal_Power_"+meter
    building_graph.add((EBU3B[meter_label], RDF.type, BRICK.Cooling_Thermal_Power_Meter))
    meter_point_of = ("EBU3B_"+meter).replace("HVAC","Chilled_Water_System")
    building_graph.add((EBU3B[meter_label], BRICK.isPointOf, EBU3B[meter_point_of]))
    
for meter in heating_thermal_power_meters:
    meter_label = "EBU3B_Heating_Thermal_Power_"+meter
    building_graph.add((EBU3B[meter_label], RDF.type, BRICK.Heating_Thermal_Power_Meter))
    meter_point_of = ("EBU3B_"+meter).replace("HVAC","Hot_Water_System")
    building_graph.add((EBU3B[meter_label], BRICK.isPointOf, EBU3B[meter_point_of]))

In [10]:
# Store output as .ttl
building_graph.serialize(destination='ebu3b_brick.ttl', format='turtle')
# Search for "EBU3B" in the output file, you'll find the instances we created.

In [11]:
#Count relationships
relationships_count = 0
res = building_graph.query("SELECT ?a ?b WHERE { ?a brickframe:isPartOf ?b .}")
for row in res:
    relationships_count += 1
res = building_graph.query("SELECT ?a ?b WHERE { ?a brickframe:isPointOf ?b .}")
for row in res:
    relationships_count += 1
res = building_graph.query("SELECT ?a ?b WHERE { ?a brickframe:feeds ?b .}")
for row in res:
    relationships_count += 1
print relationships_count

8383
